## MLZoomcamp HW6

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('car_fuel_efficiency.csv')
print(df.shape)
df.head()

(9704, 11)


,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [2]:
target = 'fuel_efficiency_mpg'

df_prep = df.copy()
df_prep = df_prep.fillna(0)

# 60/20/20 split
full_train_df, test_df = train_test_split(df_prep, test_size=0.2, random_state=1)
train_df, val_df = train_test_split(full_train_df, test_size=0.25, random_state=1)  # 0.25 * 0.8 = 0.2

print(len(train_df), len(val_df), len(test_df))

# Separate target
y_train = train_df[target].values
y_val = val_df[target].values
y_test = test_df[target].values

# Remove target from features
train_df = train_df.drop(columns=[target])
val_df = val_df.drop(columns=[target])
test_df = test_df.drop(columns=[target])

# DictVectorizer (sparse=True)
train_dicts = train_df.to_dict(orient='records')
val_dicts = val_df.to_dict(orient='records')

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

X_train.shape, X_val.shape

5822 1941 1941


((5822, 14), (1941, 14))

In [3]:
# Q1: Decision Tree (max_depth=1) and split feature

dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

feature_names = dv.get_feature_names_out()
root_feature_idx = dt.tree_.feature[0]
root_feature_name = feature_names[root_feature_idx]

# Map to raw feature among options
options_q1 = ['vehicle_weight', 'model_year', 'origin', 'fuel_type']
if '=' in root_feature_name:
    raw_name = root_feature_name.split('=')[0]
else:
    raw_name = root_feature_name

mapped_q1 = raw_name if raw_name in options_q1 else root_feature_name
root_feature_name, mapped_q1

('vehicle_weight', 'vehicle_weight')

In [4]:
# Q2: RandomForestRegressor with 10 trees -> RMSE on validation
rf10 = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf10.fit(X_train, y_train)
y_pred_val = rf10.predict(X_val)
rmse_q2 = mean_squared_error(y_val, y_pred_val, squared=False)
rmse_q2

0.4595777223092726

In [5]:
# Q3: Sweep n_estimators from 10 to 200 step 10; find first n where RMSE (rounded to 3 decimals) stops improving
from math import inf

ns = list(range(10, 201, 10))
rmses = []
for n in ns:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmses.append(mean_squared_error(y_val, y_pred, squared=False))

# Determine stopping point
best_rmse = inf
best_n = None
stop_n = None
prev_rounded = None
for n, r in zip(ns, rmses):
    r3 = round(r, 3)
    if best_n is None or r3 < round(best_rmse, 3):
        best_rmse = r
        best_n = n
        prev_rounded = r3
    else:
        # RMSE did not improve (to 3 decimals)
        stop_n = n
        break

ns, [round(r, 3) for r in rmses], best_n, round(best_rmse, 3), stop_n

([10,
  20,
  30,
  40,
  50,
  60,
  70,
  80,
  90,
  100,
  110,
  120,
  130,
  140,
  150,
  160,
  170,
  180,
  190,
  200],
 [0.46,
  0.454,
  0.452,
  0.449,
  0.447,
  0.445,
  0.445,
  0.445,
  0.445,
  0.445,
  0.444,
  0.444,
  0.444,
  0.443,
  0.443,
  0.443,
  0.443,
  0.442,
  0.442,
  0.442],
 60,
 0.445,
 70)

In [6]:
# Q4: For max_depth in [10, 15, 20, 25], compute mean RMSE across the same n_estimators sweep
from statistics import mean

max_depths = [10, 15, 20, 25]
mean_rmse_by_depth = {}
for md in max_depths:
    rmse_list = []
    for n in ns:
        rf = RandomForestRegressor(n_estimators=n, max_depth=md, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse_list.append(mean_squared_error(y_val, y_pred, squared=False))
    mean_rmse_by_depth[md] = mean(rmse_list)

best_depth = min(mean_rmse_by_depth, key=mean_rmse_by_depth.get)
{ 'mean_rmse_by_depth': {k: round(v, 4) for k, v in mean_rmse_by_depth.items()}, 'best_depth': best_depth }

{'mean_rmse_by_depth': {10: 0.4418, 15: 0.4454, 20: 0.4463, 25: 0.4459},
 'best_depth': 10}

In [7]:
# Q5: Feature importances with RF(n_estimators=10, max_depth=20)
rf_imp = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_imp.fit(X_train, y_train)
importances = rf_imp.feature_importances_
feat_names = dv.get_feature_names_out()
imp_pairs = sorted(zip(feat_names, importances), key=lambda x: x[1], reverse=True)

# Restrict to the four options
options_q5 = ['vehicle_weight', 'horsepower', 'acceleration', 'engine_displacement']
opt_importances = {name: 0.0 for name in options_q5}
for name, imp in imp_pairs:
    base = name.split('=')[0] if '=' in name else name
    if base in opt_importances:
        opt_importances[base] += imp  # accumulate if multiple one-hot columns

# Get the most important among options
most_important_q5 = max(opt_importances, key=opt_importances.get)
{ 'option_importances': {k: round(v, 4) for k, v in opt_importances.items()}, 'most_important': most_important_q5 }

{'option_importances': {'vehicle_weight': 0.9591,
  'horsepower': 0.016,
  'acceleration': 0.0115,
  'engine_displacement': 0.0033},
 'most_important': 'vehicle_weight'}

In [9]:
# Q6: XGBoost comparison for eta=0.3 vs 0.1 (100 rounds)
import xgboost as xgb

# Prepare DMatrix
feature_names = list(dv.get_feature_names_out())

# XGBoost can consume scipy CSR directly
Xtr = X_train
Xva = X_val
if hasattr(X_train, 'toarray') and not hasattr(X_train, 'dtype'):
    # keep sparse, only convert if it's some unknown type
    pass

dtrain = xgb.DMatrix(Xtr, label=y_train, feature_names=feature_names)
dval = xgb.DMatrix(Xva, label=y_val, feature_names=feature_names)

params_base = {
    'max_depth': 6,
    'min_child_weight': 1.0,
    'eta': 0.3,  # placeholder, will override
    'objective': 'reg:squarederror',
    'seed': 1,
}

results = {}
for eta in [0.3, 0.1]:
    params = params_base.copy()
    params['eta'] = eta
    model = xgb.train(params, dtrain, num_boost_round=100)
    y_pred = model.predict(dval)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    results[eta] = rmse

best_eta = min(results, key=results.get)
{ 'rmse_by_eta': {str(k): round(v, 4) for k, v in results.items()}, 'best_eta': best_eta }

{'rmse_by_eta': {'0.3': 0.4502, '0.1': 0.4262}, 'best_eta': 0.1}

## summary
- Q1: First split feature (max_depth=1 tree) → vehicle_weight
- Q2: RF (10 trees) validation RMSE → 0.4596
- Q3: RF n_estimators sweep (10..200): best rounded RMSE 0.445 at n=60; first n where RMSE (3 decimals) stops improving → 70
- Q4: Mean RMSE by max_depth: {10: 0.4418, 15: 0.4454, 20: 0.4463, 25: 0.4459}; best max_depth → 10
- Q5: Most important among [vehicle_weight, horsepower, acceleration, engine_displacement] → vehicle_weight
- Q6: XGBoost RMSEs → eta=0.3: 0.4502; eta=0.1: 0.4262; best eta → 0.1